# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

# SPARK SESSION

In [38]:
from pyspark.sql import SparkSession

In [39]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

# IMOORT DATA

In [40]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [41]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [42]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [43]:
df.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

# PREPROCESSING

In [44]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [45]:
my_cols = df.select([
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Churn'])

In [46]:
my_final_data = my_cols.na.drop()

In [47]:
my_final_data.show()

+----+--------------+-----+---------+-------------------+--------------------+-----+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|            Location|Churn|
+----+--------------+-----+---------+-------------------+--------------------+-----+
|42.0|       11066.8| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|    1|
|41.0|      11916.22|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|    1|
|38.0|      12884.75| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|    1|
|42.0|       8010.76| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|    1|
|37.0|       9191.58| 5.56|      9.0|2016-01-19 15:31:15|765 Tricia Row Ka...|    1|
|48.0|      10356.02| 5.12|      8.0|2009-03-03 23:13:37|6187 Olson Mounta...|    1|
|44.0|      11331.58| 5.23|     11.0|2016-12-05 03:35:43|4846 Savannah Roa...|    1|
|32.0|       9885.12| 6.92|      9.0|2006-03-09 14:50:20|25271 Roy Express...|    1|
|43.0|       14062.6| 5.46|     11.0|2011-09-29 05:47:23|3725 Car

In [48]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format

In [49]:
newdf = my_final_data.withColumn("Year",year(my_final_data['Onboard_Date']))

In [50]:
newdf.show()

+----+--------------+-----+---------+-------------------+--------------------+-----+----+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|            Location|Churn|Year|
+----+--------------+-----+---------+-------------------+--------------------+-----+----+
|42.0|       11066.8| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|    1|2013|
|41.0|      11916.22|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|    1|2013|
|38.0|      12884.75| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|    1|2016|
|42.0|       8010.76| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|    1|2014|
|37.0|       9191.58| 5.56|      9.0|2016-01-19 15:31:15|765 Tricia Row Ka...|    1|2016|
|48.0|      10356.02| 5.12|      8.0|2009-03-03 23:13:37|6187 Olson Mounta...|    1|2009|
|44.0|      11331.58| 5.23|     11.0|2016-12-05 03:35:43|4846 Savannah Roa...|    1|2016|
|32.0|       9885.12| 6.92|      9.0|2006-03-09 14:50:20|25271 Roy Express...|    1|2006|
|43.0|    

In [51]:
final_df = newdf.drop("Onboard_date")

In [52]:
final_df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [53]:
final_df.head()

Row(Age=42.0, Total_Purchase=11066.8, Years=7.22, Num_Sites=8.0, Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Churn=1, Year=2013)

In [54]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import split, col

In [55]:
extract_location_udf = udf(lambda x: ' '.join(x.split()[-2:]), StringType())
final_df = final_df.withColumn("Location", extract_location_udf(col("Location")))

In [56]:
final_df.show()

+----+--------------+-----+---------+-------------+-----+----+
| Age|Total_Purchase|Years|Num_Sites|     Location|Churn|Year|
+----+--------------+-----+---------+-------------+-----+----+
|42.0|       11066.8| 7.22|      8.0|     AK 89518|    1|2013|
|41.0|      11916.22|  6.5|     11.0|     RI 17756|    1|2013|
|38.0|      12884.75| 6.67|     12.0|     DE 90114|    1|2016|
|42.0|       8010.76| 6.71|     10.0|WY 30645-4695|    1|2014|
|37.0|       9191.58| 5.56|      9.0|     MH 71730|    1|2016|
|48.0|      10356.02| 5.12|      8.0|     PR 74359|    1|2009|
|44.0|      11331.58| 5.23|     11.0|IA 87713-3460|    1|2016|
|32.0|       9885.12| 6.92|      9.0|FM 59852-6150|    1|2006|
|43.0|       14062.6| 5.46|     11.0|     MA 82059|    1|2011|
|40.0|       8066.94| 7.11|     11.0|WI 51655-7561|    1|2006|
|30.0|      11575.37| 5.22|      8.0|     AA 43432|    1|2016|
|45.0|       8771.02| 6.64|     11.0|     AA 40249|    1|2015|
|45.0|       8988.67| 4.84|     11.0|     AZ 44793|    

# PIPELINE

In [57]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, VectorIndexer

In [58]:
final_df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [59]:
final_df.columns

['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Location', 'Churn', 'Year']

In [60]:
location_indexer = StringIndexer(inputCol='Location',outputCol='LocationIndex') 
location_incoder = OneHotEncoder(inputCol='LocationIndex', outputCol='LocationVec')

In [61]:
# assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Year', 'LocationVec'],
#                             outputCol='features')

assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'Year'], outputCol='features')

In [62]:
from pyspark.ml.classification import LogisticRegression

In [63]:
from pyspark.ml import Pipeline

In [64]:
log_reg = LogisticRegression(featuresCol='features', labelCol='Churn')

In [65]:
# pipeline = Pipeline(stages=[location_indexer,
#                             location_incoder,
#                             assembler,
#                             log_reg])

pipeline = Pipeline(stages=[assembler,
                            log_reg])

# SPLIT TRAIN TEST

In [66]:
final_df.show()

+----+--------------+-----+---------+-------------+-----+----+
| Age|Total_Purchase|Years|Num_Sites|     Location|Churn|Year|
+----+--------------+-----+---------+-------------+-----+----+
|42.0|       11066.8| 7.22|      8.0|     AK 89518|    1|2013|
|41.0|      11916.22|  6.5|     11.0|     RI 17756|    1|2013|
|38.0|      12884.75| 6.67|     12.0|     DE 90114|    1|2016|
|42.0|       8010.76| 6.71|     10.0|WY 30645-4695|    1|2014|
|37.0|       9191.58| 5.56|      9.0|     MH 71730|    1|2016|
|48.0|      10356.02| 5.12|      8.0|     PR 74359|    1|2009|
|44.0|      11331.58| 5.23|     11.0|IA 87713-3460|    1|2016|
|32.0|       9885.12| 6.92|      9.0|FM 59852-6150|    1|2006|
|43.0|       14062.6| 5.46|     11.0|     MA 82059|    1|2011|
|40.0|       8066.94| 7.11|     11.0|WI 51655-7561|    1|2006|
|30.0|      11575.37| 5.22|      8.0|     AA 43432|    1|2016|
|45.0|       8771.02| 6.64|     11.0|     AA 40249|    1|2015|
|45.0|       8988.67| 4.84|     11.0|     AZ 44793|    

In [67]:
train_data , test_data = final_df.randomSplit([0.7,0.3])

In [68]:
fit_model = pipeline.fit(train_data)

In [70]:
results = fit_model.transform(test_data)

# EVALUATION

In [71]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [73]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [74]:
AUC = eval.evaluate(results)

In [75]:
AUC

0.8035236396074933

# PREDICTING ON NEW DATASET

In [76]:
final_lr_model = pipeline.fit(final_df)

In [77]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [78]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [79]:
new_customers = new_customers.withColumn("Year",year(new_customers['Onboard_Date']))

In [80]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Year: integer (nullable = true)



In [81]:
test_new_customers = assembler.transform(new_customers)

In [82]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- features: vector (nullable = true)



In [83]:
test_new_customers.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+----+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|Year|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+----+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|2011|[37.0,9935.53,7.7...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|2013|[23.0,7526.94,9.2...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|2006|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:

In [87]:
final_results = final_lr_model.transform(new_customers)

In [88]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|Year|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|2011|[37.0,9935.53,7.7...|[2.42135286076803...|[0.91844114068004...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|2013|[23.0,7526.94,9.2...|[-5.9772226044143...|[0.